# SunStone 2D FDTD Demo — Young's Double Slit

This notebook runs a 2D double-slit simulation and compares the measured screen intensity to the classic interference pattern. It also loads field snapshots for visualization.

## Prerequisites
- SunStone API running locally (default: http://127.0.0.1:8000).
- `meep` available to the worker environment.
- `numpy`, `matplotlib`, `requests` installed.

In [ ]:
import json
import time
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import requests

In [ ]:
API_BASE = "http://127.0.0.1:8000"

def create_project(name: str):
    resp = requests.post(f"{API_BASE}/projects", json={"name": name})
    resp.raise_for_status()
    return resp.json()

def create_run(project_id: str, spec: dict):
    resp = requests.post(f"{API_BASE}/projects/{project_id}/runs", json={"spec": spec})
    resp.raise_for_status()
    return resp.json()

def submit_run(run_id: str, backend: str = "meep", python_executable: str | None = None):
    payload = {"mode": "local", "backend": backend}
    if python_executable:
        payload["python_executable"] = python_executable
    resp = requests.post(f"{API_BASE}/runs/{run_id}/submit", json=payload)
    resp.raise_for_status()
    return resp.json()

def get_run(run_id: str):
    resp = requests.get(f"{API_BASE}/runs/{run_id}")
    resp.raise_for_status()
    return resp.json()

def list_artifacts(run_id: str):
    resp = requests.get(f"{API_BASE}/runs/{run_id}/artifacts")
    resp.raise_for_status()
    return resp.json()["artifacts"]

def download_artifact(run_id: str, path: str, dest: Path):
    resp = requests.get(f"{API_BASE}/runs/{run_id}/artifacts/{path}")
    resp.raise_for_status()
    dest.write_bytes(resp.content)
    return dest

In [ ]:
# Double-slit parameters (meters)
cell_x = 20e-6
cell_y = 12e-6
pml = 2e-6
resolution = 40

slit_width = 1.0e-6
slit_sep = 3.0e-6
barrier_thickness = 0.5e-6

# Frequency (Hz) ~ 1 micron wavelength in vacuum
center_freq = 3.0e14
fwidth = 0.02 * center_freq

# Barrier blocks to form two slits (PEC)
half_h = cell_y / 2
top_height = half_h - (slit_sep / 2 + slit_width / 2)
mid_height = slit_sep - slit_width
bottom_height = top_height

top_center = (slit_sep / 2 + slit_width / 2 + top_height / 2)
mid_center = 0.0
bottom_center = -(slit_sep / 2 + slit_width / 2 + bottom_height / 2)

pec_blocks = [
    {"type": "block", "size": [barrier_thickness, top_height, 0], "center": [0, top_center, 0], "material": "pec"},
    {"type": "block", "size": [barrier_thickness, mid_height, 0], "center": [0, mid_center, 0], "material": "pec"},
    {"type": "block", "size": [barrier_thickness, bottom_height, 0], "center": [0, bottom_center, 0], "material": "pec"},
]

source = {
    "type": "gaussian_pulse",
    "center_freq": center_freq,
    "fwidth": fwidth,
    "component": "Ez",
    "position": [-0.45 * cell_x, 0, 0],
    "size": [0, 0.9 * cell_y, 0],
}

outputs = {
    "field_movie": {
        "components": ["Ez", "Hz"],
        "dt": 2e-15,
        "start_time": 0.0,
        "stop_time": 2e-13,
        "center": [0, 0, 0],
        "size": [cell_x, cell_y, 0],
    },
    "field_snapshot": {
        "components": ["Ez"],
        "center": [0, 0, 0],
        "size": [cell_x, cell_y, 0],
    }
}

spec = {
    "version": "0.1",
    "units": {"length": "m", "time": "s", "frequency": "Hz"},
    "domain": {"cell_size": [cell_x, cell_y, 0], "resolution": resolution, "symmetry": [], "dimension": "2d"},
    "boundary_conditions": {"type": "pml", "pml_thickness": [pml, pml, 0]},
    "materials": {
        "vac": {"model": "constant", "eps": 1.0},
        "pec": {"model": "pec", "eps": 1.0},
    },
    "geometry": pec_blocks,
    "sources": [source],
    "monitors": [],
    "run_control": {"until": "time", "max_time": 2e-13},
    "resources": {"mode": "local"},
    "outputs": outputs,
}

spec

In [ ]:
project = create_project("double-slit-2d")
run = create_run(project["id"], spec)
submit_run(run["id"], backend="meep")

# Poll run status
for _ in range(60):
    status = get_run(run["id"])
    print(status["status"])
    if status["status"] in ("succeeded", "failed"):
        break
    time.sleep(2)

In [ ]:
artifacts = list_artifacts(run["id"])
paths = [a["path"] for a in artifacts]
[p for p in paths if "field_movie.npz" in p or "field_snapshot.npz" in p]

In [ ]:
movie_path = download_artifact(run["id"], "outputs/fields/field_movie.npz", Path("field_movie.npz"))
data = np.load(movie_path)
times = data["times"]
Ez = data["Ez"]  # shape: (frames, ny, nx)

plt.figure(figsize=(8, 4))
plt.imshow(Ez[-1], cmap="RdBu", origin="lower")
plt.colorbar(label="Ez")
plt.title("Ez field (final frame)")
plt.tight_layout()
plt.show()

In [ ]:
ny, nx = Ez.shape[1], Ez.shape[2]
screen_x = int(0.92 * nx)
window = Ez[-20:] if Ez.shape[0] > 20 else Ez
intensity = np.mean(window[:, :, screen_x] ** 2, axis=0)

y = np.linspace(-cell_y / 2, cell_y / 2, ny)

wavelength = 3e8 / center_freq
d = slit_sep
a = slit_width
L = 0.45 * cell_x
theta = np.arctan2(y, L)
beta = np.pi * a * np.sin(theta) / wavelength
delta = np.pi * d * np.sin(theta) / wavelength
I_theory = (np.sinc(beta / np.pi) ** 2) * (np.cos(delta) ** 2)
I_theory /= I_theory.max()

I_sim = intensity / intensity.max()

plt.figure(figsize=(8, 4))
plt.plot(y * 1e6, I_sim, label="Simulation")
plt.plot(y * 1e6, I_theory, label="Theory", linestyle="--")
plt.xlabel("Screen Y (µm)")
plt.ylabel("Normalized intensity")
plt.title("Double-slit interference pattern")
plt.legend()
plt.tight_layout()
plt.show()